In [1]:
import networkx as nx
import seaborn
import custom_funcs as cf
import matplotlib.pyplot as plt
import numpy as np

from joblib import Parallel, delayed
from time import time

%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
# Set Seaborn styles & context
seaborn.set_context("paper")
seaborn.set_style('white')

In [3]:
# Read in data and do data pre-processing
G = nx.read_gpickle('20150902_all_ird Final Graph.pkl')
G = cf.impute_reassortant_status(G)
G = cf.impute_host_group_name(G)
G = cf.impute_weights(G)

In [4]:
# Compute proportion reassortant
data_props = cf.edge_proportion_reassortant(G, attr='host_group', exclusions=['Unknown'])
data_props

{'diff_attr': 0.8131267590611879, 'same_attr': 0.37525262774838164}

In [5]:
start = time()
def null_distribution_proportion_reassortant(G):
    G_shuffled = cf.shuffle_node_attribute_label(G, 'host_group')
    props = cf.edge_proportion_reassortant(G_shuffled, 'host_group', exclusions=['Unknown'])
    return props

props_null = Parallel(n_jobs=-1)(delayed(null_distribution_proportion_reassortant)(G) for i in range(20))
end = time()
print(end - start)

201.92485189437866


In [6]:
null = pd.DataFrame(props_null)
null.mean()

Traceback (most recent call last):
  File "/Users/ericmjl/anaconda/lib/python3.4/site-packages/IPython/core/interactiveshell.py", line 3066, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-32deb010c61d>", line 1, in <module>
    null = pd.DataFrame(props_null)
NameError: name 'pd' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/ericmjl/anaconda/lib/python3.4/site-packages/IPython/core/interactiveshell.py", line 1877, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/ericmjl/anaconda/lib/python3.4/site-packages/IPython/core/ultratb.py", line 970, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/Users/ericmjl/anacon

ERROR: Internal Python error in the inspect module.
Below is the traceback from this internal error.


Unfortunately, your original traceback can not be constructed.



TypeError: 'NoneType' object is not iterable

In [ ]:
data = pd.DataFrame([data_props])
data.mean()

In [ ]:
fig = plt.figure(figsize=(3,2))
ax = fig.add_subplot(1,1,1)
ind = np.arange(2)
width = 0.35

ax.bar(ind, null.mean(), width, color='blue', label='Null', alpha=0.5, yerr=null.std()*5)
ax.bar(ind+width, data.mean(), width, color='blue', label='Data')
ax.set_xticks(ind+width)
ax.set_xticklabels(['Different', 'Same'])
ax.set_xlabel('Hosts Pair')
ax.set_ylabel('Proportion Reassortant')
ax.legend()
ax.set_ylim(0,1)

In [ ]:
# Between which host groups are they over-represented or under-represented?

def host_group_domain_graph(G):
    """
    Computes the proportion of reassortant representation when crossing between host groups.
    """
    hg_graph = nx.DiGraph()

    for n, node_d in G.nodes(data=True):

        in_edges = G.in_edges(n, data=True)
        total_edges = len(in_edges)
        is_reassortant = node_d['reassortant']

        sk_hg = G.node[n]['host_group']
        if sk_hg not in hg_graph.nodes():
            hg_graph.add_node(sk_hg)

        for sc, _, edge_d in in_edges:
            sc_hg = G.node[sc]['host_group']
            if sc_hg not in hg_graph.nodes():
                hg_graph.add_node(sc_hg)

            if (sc_hg, sk_hg) not in hg_graph.edges():
                hg_graph.add_edge(sc_hg, sk_hg, total=edge_d['weight'], reassortant=0)
            if (sc_hg, sk_hg) in hg_graph.edges():
                hg_graph.edge[sc_hg][sk_hg]['total'] += edge_d['weight']

            if is_reassortant:
                hg_graph.edge[sc_hg][sk_hg]['reassortant'] += edge_d['weight']

    for sc, sk, d in hg_graph.edges(data=True):
        hg_graph.edge[sc][sk]['p_reassortant'] = d['reassortant'] / d['total']
        
    return hg_graph


In [ ]:
hg_graph = host_group_domain_graph(G)
hg_graph.edges(data=True)

In [ ]:
def null_proportion_hg_reassortant(G, equally=False):
    
    G_shuffled = cf.shuffle_node_attribute_label(G, 'host_group', equally)
    hg_graph_shuf = host_group_domain_graph(G_shuffled)
    
    return hg_graph_shuf

In [ ]:
start = time()
results = Parallel(n_jobs=-1)(delayed(null_proportion_hg_reassortant)(G, equally=True) for i in range(20))
len(results)
end = time()
print(end - start)

In [ ]:
def distr_null_p_reassortant(list_of_hg_graphs):
    hg_graph = nx.DiGraph()
    for g in list_of_hg_graphs:
        hg_graph.add_nodes_from(g.nodes())
        for sc, sk, d in g.edges(data=True):
            if (sc, sk) not in hg_graph.edges():
                hg_graph.add_edge(sc, sk, p_reassortant=[d['p_reassortant']])
            else:
                hg_graph.edge[sc][sk]['p_reassortant'].append(d['p_reassortant'])
    return hg_graph

summaryG = distr_null_p_reassortant(results)

In [ ]:
summaryG.edges(data=True)

In [ ]:
name_map = {'Wild':'W',
            'Domestic':'D',
            'Human':'H'}
names = []
means = []
stds  = []
for sc, sk, d in summaryG.edges(data=True):
    if sc == 'Unknown' or sk == 'Unknown':
        pass
    else:
        mean = np.mean(d['p_reassortant'])
        std = np.std(d['p_reassortant'])
        
        names.append('{0}:{1}'.format(name_map[sc], name_map[sk]))
        means.append(mean)
        stds.append(std)

In [ ]:
data = []

for sc, sk, d in hg_graph.edges(data=True):
    if sc == 'Unknown' or sk == 'Unknown':
        pass
    else:
        data.append(d['p_reassortant'])
data

In [ ]:
fig = plt.figure(figsize=(4,2))

ind = np.arange(len(means))
width = 0.35

ax = fig.add_subplot(1,1,1)
ax.bar(ind, means, width=width, color='blue', label='Null', yerr=np.array(stds)*5, alpha=0.5)
ax.bar(ind+width, data, width=width, color='blue', label='Data')
ax.set_xticks(ind+width)
ax.set_xticklabels(names)
ax.set_ylabel('Weighted Proportion\nReassortant Per Event')
ax.set_xlabel('Host Group Transition')
ax.legend(loc='upper center')
plt.subplots_adjust(left=0.15, right=0.95)
plt.savefig('figures/Proportion Reassortant Ecological.pdf')
plt.show()

In [274]:
np.array(means) - np.array(data)

array([ 0.05333104, -0.4767556 , -0.14965754, -0.29672359,  0.1340915 ,
       -0.53273047, -0.33293242, -0.55757043, -0.0677194 ])